# Word2Vec ile Duygu Analizi

### Egzersiz hedefleri:
- Kelimeleri Word2Vec ile vektörlere dönüştürme
- Word2vec tarafından verilen kelime temsilini RNN'e besleme

<hr>

▶️ Bu hücreyi çalıştırın ve kullandığınız 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) sürümünün ≥ 4.0 olduğundan emin olun!

In [1]:
!pip freeze | grep gensim

gensim==4.3.3


# Veri


❓ **Soru** ❓ Önce veriyi yükleyelim. Fonksiyonun içinde neler olduğunu anlamanıza gerek yok, burada önemli değil.

⚠️ **Uyarı** ⚠️ `load_data` fonksiyonunun `percentage_of_sentences` argümanı var. Bilgisayarınıza bağlı olarak, çok fazla cümle bilgisayarınızın yavaşlamasına hatta donmasına neden olabilir - RAM'iniz taşabilir. Bu nedenle, **cümlelerin %10'u ile başlamalı** ve bilgisayarınızın bunu kaldırıp kaldırmadığını görmelisiniz. Aksi takdirde daha düşük bir sayı ile tekrar çalıştırın.

⚠️ **UYARI** ⚠️ **_En büyük RAM'e sahip kim_ oyununu oynamaya gerek yok!** Amaç, prototip yapmak için modellerinizi hızlıca çalıştırmaktır. Gerçek hayatta bile, hızlıca döngü yapıp hata ayıklamak için verilerinizin bir alt kümesi ile başlamanız önerilir. Bu nedenle sayıyı sadece en iyi doğruluğu elde etmek istiyorsanız artırın.

In [2]:
import numpy as np

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2026-02-15 13:05:12.532079: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 13:05:12.771145: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 13:05:13.076754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-15 13:05:13.379188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-15 13:05:13.381546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-15 13:05:13.834596: I tensorflow/core/platform/cpu_feature_guard.cc:

Önceki egzersizde, bir Word2vec temsili eğittiniz ve tüm eğitim cümlelerinizi RNN'e beslemek için dönüştürdünüz. Bu şeklin ilk adımında gösterildiği gibi:

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/06-DL/NLP/word2vec_representation.png" alt="Word2Vec with RNN" width="400px" />



❓ **Soru** ❓ Burada, önceki egzersizde yaptığınızın aynısını tekrar yapalım. İlk olarak, eğitim cümleleriniz üzerinde bir word2vec modeli eğitin (istediğiniz argümanlarla). Bunu `word2vec` değişkeninde saklayın.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=10, window=10)

Eğitim ve test verilerinizi RNN'e besleyebileceğiniz bir şeye dönüştürmek için önceki egzersizin fonksiyonlarını yeniden kullanalım.

❓ **Soru** ❓ Aşağıdaki fonksiyonu okuyun, ne olup bittiğinden emin olun ve çalıştırın.

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ Çalıştığından emin olmak için, `X_train_pad` ve `X_test_pad` için aşağıdakileri kontrol edelim:
- numpy dizileri olmalılar
- 3-boyutlu olmalılar
- son boyut word2vec gömüleme uzayınızın boyutu kadar olmalı (`word2vec.wv.vector_size` ile alabilirsiniz)
- ilk boyut `X_train` ve `X_test`'inizin boyutu kadar olmalı

✅ **İyi Uygulama** ✅ Bu tür testler oldukça önemli! Sadece bu egzersizde değil, gerçek hayat uygulamalarında da. Hataları çok geç keşfetmeyi ve tüm notebook boyunca yayılmalarını önler.

In [6]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

Kendi modelinizi test edebileceğiniz çok basit bir modele sahip olmak her zaman iyidir - çok basit bir algoritmadan daha iyi bir şey yaptığınızdan emin olmak için.

❓ **Soru** ❓ Baseline doğruluğunuz nedir? Bu durumda, baseline'ınız `y_train`'de en çok bulunan etiketi tahmin etmek olabilir (tabii ki, veri seti dengeliyse, baseline doğruluğu 1/n'dir, burada n sınıf sayısıdır - burada 2).

In [7]:
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1

print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Number of labels in train set {0: 1265, 1: 1235}
Baseline accuracy:  0.492


# Model

❓ **Soru** ❓ Aşağıdaki katmanları içeren bir RNN yazın:
- bir `Masking` katmanı
- 20 birim ve `tanh` aktivasyon fonksiyonu ile bir `LSTM`
- 10 birim ile bir `Dense`
- görevinize bağlı bir çıktı katmanı

Sonra, modelinizi derleyin (optimizer olarak `rmsprop` kullanmanızı tavsiye ederiz - en azından başlangıç için)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    return model

model = init_model()

❓ **Soru** ❓ Modeli gömülmüş ve dolgulanmış verileriniz üzerinde eğitin - early stopping kriterini unutmayın.

❗ **Not** ❗ Doğruluğunuz büyük ölçüde eğitim corpus'unuza bağlı olacak. Burada sadece performansınızın baseline modelin üzerinde olduğundan emin olun (bu, başlangıçtaki IMDB verilerinin sadece %20'sini yüklemiş olsanız bile böyle olmalı).

In [9]:
#X_train_pad_short = X_train_pad[:500] # These two lines are just to accelerate the cell run
#y_train_short = y_train[:500]

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train,
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 21s 207ms/step - accuracy: 0.5189 - loss: 0.6941 - val_accuracy: 0.5387 - val_loss: 0.6890
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 12s 213ms/step - accuracy: 0.5697 - loss: 0.6819 - val_accuracy: 0.5813 - val_loss: 0.6802
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 206ms/step - accuracy: 0.5971 - loss: 0.6705 - val_accuracy: 0.5720 - val_loss: 0.6804
Epoch 4/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 9s 157ms/step - accuracy: 0.6074 - loss: 0.6582 - val_accuracy: 0.5800 - val_loss: 0.6813
Epoch 5/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 204ms/step - accuracy: 0.6429 - loss: 0.6443 - val_accuracy: 0.6093 - val_loss: 0.6605
Epoch 6/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 198ms/step - accuracy: 0.6446 - loss: 0.6334 - val_accuracy: 0.6213 - val_loss: 0.6554
Epoch 7/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 173ms/step - accuracy: 0.6463 - loss: 0.6229 - val_accuracy: 0.5853 - val_loss: 0.6585
Epoch 8/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 12s 214ms/step - accuracy: 0.6754 - loss: 0.6097 - v

❓ **Soru** ❓ Modelinizi test seti üzerinde değerlendirin

In [10]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 65.920%


# Eğitilmiş Word2Vec - Transfer Learning

Doğruluğunuz baseline modelin üzerinde olmasına rağmen oldukça düşük olabilir. Veri temizleme ve gömülemenin kalitesini artırma gibi bunu iyileştirmek için birçok seçenek var.

Burada veri temizleme stratejilerine girmeyeceğiz. Gömülememizin kalitesini artırmaya çalışalım. Ama sadece daha büyük bir corpus yüklemek yerine, başkalarının öğrendiği gömülemeden neden faydalanmayalım? Çünkü bir gömülemenin kalitesi, yani kelimelerin yakınlığı, farklı görevlerden türetilebilir. Transfer learning tam olarak budur.



❓ **Soru** ❓ Word2vec'te mevcut olan tüm farklı modelleri listeleyin: 

In [11]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ Modellerin listesini ve boyutlarını [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models)'de de bulabilirsiniz.

❓ **Soru** ❓ Önceden eğitilmiş word2vec gömüleme uzaylarından birini yükleyin.

Bunu `api.load(seçtiğiniz-model)` ile yapabilir ve `word2vec_transfer`'da saklayabilirsiniz

<details>
    <summary>💡 İpucu</summary>
    
`glove-wiki-gigaword-50` modeli daha küçük olduğu için (65 MB) başlamak için iyi bir aday.

</details>

In [12]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


❓ **Soru** ❓ Vocabulary'nin boyutunu kontrol edin, ayrıca gömüleme uzayının boyutunu da kontrol edin.

In [13]:
print(len(word2vec_transfer.key_to_index))
print(len(word2vec_transfer['dog']))

400000
50


❓ `X_train` ve `X_test`'i gömleyelim, fonksiyonları sağladığımız ilk soruda olduğu gibi! (`embed_sentence_with_TF` fonksiyonunda derinlemesine girmeyeceğimiz küçük bir fark var)

In [14]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Soru** ❓ Sonuçlarınızı dolgulamayı unutmayın ve bunları `X_train_pad_2` ve `X_test_pad_2`'de saklayın.

In [15]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

❓ **Soru** ❓ Bir modeli yeniden başlatın ve yeni gömülmüş (ve dolgulanmış) verileriniz üzerinde eğitin! Test setinizde değerlendirin ve önceki doğruluğunuzla karşılaştırın.

❗ **Not** ❗ Buradaki eğitim biraz zaman alabilir. Sadece 10 epoch hesaplayabilirsiniz (bu **iyi** bir uygulama değil, sadece çok uzun beklememek için) ve eğitim devam ederken bir sonraki egzersize geçebilir - ya da mola verebilirsiniz, muhtemelen hak ediyorsunuz ;)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()

model.fit(X_train_pad_2, y_train,
          batch_size = 32,
          epochs=10,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 16s 182ms/step - accuracy: 0.5171 - loss: 0.6967 - val_accuracy: 0.5547 - val_loss: 0.6840
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 175ms/step - accuracy: 0.5954 - loss: 0.6764 - val_accuracy: 0.6027 - val_loss: 0.6660
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 207ms/step - accuracy: 0.6200 - loss: 0.6528 - val_accuracy: 0.6160 - val_loss: 0.6483
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - accuracy: 0.6457 - loss: 0.6273 - val_accuracy: 0.6760 - val_loss: 0.6150
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 190ms/step - accuracy: 0.6754 - loss: 0.6000 - val_accuracy: 0.6893 - val_loss: 0.5830
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - accuracy: 0.7057 - loss: 0.5759 - val_accuracy: 0.6533 - val_loss: 0.6386
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 175ms/step - accuracy: 0.7086 - loss: 0.5658 - val_accuracy: 0.7480 - val_loss: 0.5425
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 12s 211ms/step - accuracy: 0.7343 - loss: 0.5547 - val_accu

In [17]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 72.640%


Yeni word2vec'iniz büyük bir corpus üzerinde eğitildiği için, çok çok fazla kelime için temsile sahip! Küçük veri setinizle olduğundan çok daha fazla, özellikle eğitim setinde belirli bir sayıdan fazla bulunmayan kelimeleri attığınız için. Bu nedenle, eğitim ve test setinizde çok daha fazla gömülmüş kelime var, bu da her iterasyonu öncekinden daha uzun hale getiriyor